In [ ]:
# phase vocoder example
# (c) V Lazzarini, 2010
# GNU Public License
#

- https://dsp.stackexchange.com/questions/1232/programming-a-vocoder
- https://audioprograming.wordpress.com/2012/03/02/a-phase-vocoder-in-python

In [ ]:
%pylab inline
rcParams['figure.figsize'] = (15, 4)

In [ ]:
from scipy.io import wavfile
from IPython.display import Audio
from scipy.fftpack import dct

In [ ]:
sampleRate, signal = wavfile.read('media/misery.wav')
signal.shape

In [ ]:
subplot(121)
plot(signal)
subplot(122)
plot(log(abs(fft.rfft(signal))))
Audio(signal, rate=sampleRate)

In [ ]:
N = 2048
H = N // 4
L = len(signal)
tscale = 0.45

In [ ]:
# signal blocks for processing and output
phi = zeros(N)
out = zeros(N, dtype=complex)
output = zeros(int(L / tscale + N) + 1)

amp = max(signal)
window = hanning(N)
p = 0
pp = 0

while p < L-(N+H):
    
    # take the spectra of two consecutive windows
    p1 = int(p)
    spec1 =  fft.fft(window * signal[p1:p1 + N])
    spec2 =  fft.fft(window * signal[p1 + H:p1 + N + H])
    
    # take their phase difference and integrate
    phi += (angle(spec2) - angle(spec1))

    # bring the phase back to between pi and -pi
    for i in phi:
        while i < -pi:
            i += 2 * pi
        while i >= pi:
            i -= 2 * pi
    out.real, out.imag = cos(phi), sin(phi)

    # make the output FFT with frequency magnitudes from the next spectrum
    # but use the unrolled, more accurate phase data
    synthesized = abs(spec2) * out
    
    # inverse FFT and overlap-add
    output[pp:pp + N] += window * real(fft.ifft(synthesized))
    
    # move the output pointer forward by the hopesize
    pp += H
    
    # move the signal pointer forward by the hopesize scaled
    p += H * tscale

In [ ]:
#  write file to output, scaling it to original amp
#data = array(amp * output / max(output)).astype('int16')
#wavfile.write('clip_pv.wav', sampleRate, data)

In [ ]:
Audio(data=output, rate=sampleRate)

In [ ]:
subplot(121)
plot(output)
subplot(122)
plot(log(abs(fft.rfft(output))))